In [1]:
import os
import numpy as np
import pandas as pd
import warnings

# This is not recommended but I am doing this to suppress warnings from SARIMAX
warnings.simplefilter('ignore')

countryName = 'Iran'

nFeatures = 1

nDaysMin = 6
k = 5

nValid = 10
nTest = 10

dataDir = os.path.join('data', 'JHU', 'upto07082020_forPublication')


# confirmedFilename = 'time_series_covid19_confirmed_global.csv'
# deathsFilename = 'time_series_covid19_deaths_global.csv'
# recoveredFilename = 'time_series_covid19_recovered_global.csv'

confirmedFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_confirmed_global.csv'
deathsFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_deaths_global.csv'
recoveredFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_recovered_global.csv'

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps, k):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix + k >= len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:end_ix+k]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
def meanAbsolutePercentageError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    absPcErrorList = [absError/yTrue for absError, yTrue in zip(absErrorList, yTrueList)]
    MAPE = 100*np.mean(absPcErrorList)
    return MAPE

def meanAbsolutePercentageError_kDay(yTrueListList, yPredListList):
    # Store true and predictions for day 1 in a list, day 2 in a list and so on
    # Keep each list of these lists in a respective dict with key as day #
    yTrueForDayK = {}
    yPredForDayK = {}
    for i in range(len(yTrueListList[0])):
        yTrueForDayK[i] = []
        yPredForDayK[i] = []
    for yTrueList, yPredList in zip(yTrueListList, yPredListList):
        for i in range(len(yTrueList)):
            yTrueForDayK[i].append(yTrueList[i])
            yPredForDayK[i].append(yPredList[i])
            
    # Get MAPE for each day in a list
    MAPEList = []
    for i in yTrueForDayK.keys():
        MAPEList.append(meanAbsolutePercentageError(yTrueForDayK[i], yPredForDayK[i]))
    return np.mean(MAPEList)

def meanForecastError(yTrueList, yPredList):
    forecastErrors = [yTrue - yPred for yTrue, yPred in zip(yTrueList, yPredList)]
    MFE = np.mean(forecastErrors)
    return MFE

def meanAbsoluteError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(absErrorList)

def meanSquaredError(yTrueList, yPredList):
    sqErrorList = [np.square(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(sqErrorList)

def rootMeanSquaredError(yTrueList, yPredList):
    return np.sqrt(meanSquaredError(yTrueList, yPredList))

def medianSymmetricAccuracy(yTrueList, yPredList):
    '''https://helda.helsinki.fi//bitstream/handle/10138/312261/2017SW001669.pdf?sequence=1'''
    logAccRatioList = [np.abs(np.log(yPred/yTrue)) for yTrue, yPred in zip(yTrueList, yPredList)]
    MdSA = 100*(np.exp(np.median(logAccRatioList))-1)
    return MdSA

def medianSymmetricAccuracy_kDay(yTrueListList, yPredListList):
    # Store true and predictions for day 1 in a list, day 2 in a list and so on
    # Keep each list of these lists in a respective dict with key as day #
    yTrueForDayK = {}
    yPredForDayK = {}
    for i in range(len(yTrueListList[0])):
        yTrueForDayK[i] = []
        yPredForDayK[i] = []
    for yTrueList, yPredList in zip(yTrueListList, yPredListList):
        for i in range(len(yTrueList)):
            yTrueForDayK[i].append(yTrueList[i])
            yPredForDayK[i].append(yPredList[i])
    # Get MdSA for each day in a list
    MdSAList = []
    for i in yTrueForDayK.keys():
        MdSAList.append(medianSymmetricAccuracy(yTrueForDayK[i], yPredForDayK[i]))
    return(np.mean(MdSAList))

In [4]:
# Function to get all three frames for a given country
def getCountryCovidFrDict(countryName):
    countryCovidFrDict = {}
    for key in covidFrDict.keys():
        dataFr = covidFrDict[key]
        countryCovidFrDict[key] = dataFr[dataFr['Country/Region'] == countryName]
    return countryCovidFrDict

In [5]:
# Load all 3 csv files
covidFrDict = {}
# covidFrDict['confirmed'] = pd.read_csv(os.path.join(dataDir, confirmedFilename))
# covidFrDict['deaths'] = pd.read_csv(os.path.join(dataDir, deathsFilename))
covidFrDict['confirmed'] = pd.read_csv(confirmedFilename)
covidFrDict['deaths'] = pd.read_csv(deathsFilename)

# Recovered is back again!
covidFrDict['recovered'] = pd.read_csv(recoveredFilename)

countryCovidFrDict = getCountryCovidFrDict(countryName)

# Get list of dates
colNamesList = list(countryCovidFrDict['confirmed'])
dateList = [colName for colName in colNamesList if '/20' in colName]
dataList = [countryCovidFrDict['confirmed'][date].iloc[0] for date in dateList]
dataDict = dict(zip(dateList, dataList))

# Get time series for cases > 100 only
daysSince = 100
nCasesGreaterDaysSinceList = []
datesGreaterDaysSinceList = []

for key in dataDict.keys():
    if dataDict[key] > daysSince:
        datesGreaterDaysSinceList.append(key)
        nCasesGreaterDaysSinceList.append(dataDict[key])
        
XList, yList = split_sequence(nCasesGreaterDaysSinceList, nDaysMin, k)

XTrainList = XList[0:len(XList)-(nValid + nTest)]
XValidList = XList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
XTestList = XList[-nTest:]

yTrain = yList[0:len(XList)-(nValid + nTest)]
yValid = yList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
yTest = yList[-nTest:]

print('Total size of data points for LSTM:', len(yList))
print('Size of training set:', len(yTrain))
print('Size of validation set:', len(yValid))
print('Size of test set:', len(yTest))

# Convert from list to matrix
XTrain = XTrainList.reshape((XTrainList.shape[0], XTrainList.shape[1], nFeatures))
XValid = XValidList.reshape((XValidList.shape[0], XValidList.shape[1], nFeatures))
XTest = XTestList.reshape((XTestList.shape[0], XTestList.shape[1], nFeatures))

Total size of data points for LSTM: 123
Size of training set: 103
Size of validation set: 10
Size of test set: 10


# Vanilla LSTM

In [6]:
# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# nNeurons = 100
# # define model
# model = Sequential()
# model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
# model.add(Dense(1))
# opt = Adam(learning_rate=0.1)
# model.compile(optimizer=opt, loss='mse')

# XTestNew = XTest.copy()
# for day in range(k):
#     print('Day:', day)
#     yPred = model.predict(XTestNew, verbose=0)
#     for i in range(len(yPred)):
#         yPredListList[i].append(yPred[i][0])
#     print('Prediction:', yPred)
#     XTestNew = np.delete(XTestNew, 0, axis=1)
#     yPred = np.expand_dims(yPred, 2)
#     XTestNew = np.append(XTestNew, yPred, axis=1)

In [7]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

nNeurons = 100
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    
    # define model
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 0.15265948489813125
Updating best MAPE to 0.15265948489813125...
Updating best seed to 0...
1 1.525750614988043
2 2.7432324814369378
3 0.910890383733187
4 99.81460446186188
5 2.2397527903458196
6 0.7840556285256327
7 0.9941671564151404
8 99.80810004723119
9 0.27696643799408593
10 1.2524175435623266
11 0.6410690462847807
12 2.4911449604951956
13 36.04966757357327
14 0.8887264988509453
15 0.3854363099369211
16 99.85084276705926
17 2.6491430063087735
18 0.6433123250543046
19 5.692360612585585
20 0.1536146723220842
21 1.475126379061786
22 99.85403891528037
23 0.7172235836259128
24 0.8661343292351722
25 2.0296336177932934
26 1.4615239482180626
27 1.5983911155441377
28 0.3017874878490831
29 99.80609814668108
30 0.22085856543435267
31 99.80965596454293
32 0.2799968503434972
33 99.9645355817506
34 0.45063974568403653
35 0.5454843114225334
36 1.1697374657351294
37 99.8077894576533
38 1.223707332605245
39 279.4256315623333
40 1.3024594614996352
41 9.659242397822755
42 0.7630596739868325
43 4.2

# Stacked LSTM

In [8]:
# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 61.880554959695836
Updating best MAPE to 61.880554959695836...
Updating best seed to 0...
1 6.904140244289242
Updating best MAPE to 6.904140244289242...
Updating best seed to 1...
2 2.0184048211730548
Updating best MAPE to 2.0184048211730548...
Updating best seed to 2...
3 56.173631904934744
4 57.32698084218515
5 9.311836636567316
6 99.80860574885196
7 63.210456880476464
8 98.18478556162748
9 97.0072279473803
10 0.43286739706399474
Updating best MAPE to 0.43286739706399474...
Updating best seed to 10...
11 61.05056602580525
12 62.17186596803043
13 63.11374609459999
14 58.14999494312112
15 8.833110603784458
16 1.2081280163954955
17 59.96628800133341
18 55.96088737227207
19 58.83502135803101
20 2.2509766900101082
21 1.0221305699316683
22 58.857515542010766
23 1.9180215595210484
24 0.754862630326073
25 60.84386447753995
26 0.5055332399632436
27 2.691118108239612
28 59.23248387564304
29 60.241105578301394
30 2.1946782020656634
31 58.89159659188077
32 99.97254599777867
33 58.232039782112


# Bidirectional LSTM

In [11]:
from tensorflow.keras.layers import Bidirectional

# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 1.1147762098782674
Updating best MAPE to 1.1147762098782674...
Updating best seed to 0...
1 1.005969178613074
Updating best MAPE to 1.005969178613074...
Updating best seed to 1...
2 1.810222894267163
3 2.9881158834249004
4 1.3474641163913232
5 0.8706951777205422
Updating best MAPE to 0.8706951777205422...
Updating best seed to 5...
6 99.81781943142146
7 0.9456673678140562
8 99.81564109019612
9 2.844397046717819
10 3.1147715250853465
11 99.82462255794073
12 0.7020513367752446
Updating best MAPE to 0.7020513367752446...
Updating best seed to 12...
13 99.86888181274198
14 6.041697707886776
15 99.83836885373532
16 1.8341125236270082
17 3.8476574218928667
18 1.656963672026992
19 0.9363113968682557
20 0.5911040018048197
Updating best MAPE to 0.5911040018048197...
Updating best seed to 20...
21 99.88061131588356
22 2.905807766089145
23 0.932870068198263
24 1.0960220996290773
25 2.803431144870983
26 17.118385090905157
27 2.5287635254064775
28 0.6017914693651001
29 0.7117869475099358
30 0.548

# CNN LSTM

In [14]:
# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import Flatten
# from tensorflow.compat.v1.keras.layers import TimeDistributed
# from tensorflow.keras.layers import Conv1D
# from tensorflow.keras.layers import MaxPooling1D

# nSeq = 5
# nSteps = 3
# nNeurons = 50
# nFeatures = 1
# nFilters = 64

# XTestListNew = XTestList.copy()
# print(XTestListNew.shape)

# model = Sequential()
# model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
# model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
# model.add(TimeDistributed(Flatten()))
# model.add(LSTM(nNeurons, activation='relu'))
# model.add(Dense(1))
# opt = Adam(learning_rate=0.1)
# model.compile(optimizer=opt, loss='mse')

# XTestCNN = XTestListNew.reshape((XTestListNew.shape[0], nSeq, nSteps, nFeatures))
# yPred = model.predict(XTestCNN, verbose=0)

# XTestListNew = np.delete(XTestListNew, 0, axis=1)
# XTestListNew = np.append(XTestListNew, yPred, axis=1)

In [17]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.compat.v1.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

# Number of subsequences to break X into (we do 15 = 5x3, 5 subsequences of size 3 each)
nSeq = 3
nSteps = 2

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainCNN = XTrainList.reshape((XTrainList.shape[0], nSeq, nSteps, nFeatures))

# print(XTrainCNN.shape)
# print(XValidCNN.shape)
# print(XTestCNN.shape)

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainCNN, yTrain[:,0], epochs=1000, verbose=0)
    
    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidListNew = XValidList.copy()
    for day in range(k):
        XValidCNN = XValidListNew.reshape((XValidListNew.shape[0], nSeq, nSteps, nFeatures))
        yPred = model.predict(XValidCNN, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidListNew = np.delete(XValidListNew, 0, axis=1)
        XValidListNew = np.append(XValidListNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainCNN, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestListNew = XTestList.copy()
for day in range(k):
    XTestCNN = XTestListNew.reshape((XTestListNew.shape[0], nSeq, nSteps, nFeatures))
    yPred = model.predict(XTestCNN, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestListNew = np.delete(XTestListNew, 0, axis=1)
    XTestListNew = np.append(XTestListNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 0.21921060322621747
Updating best MAPE to 0.21921060322621747...
Updating best seed to 0...
1 0.5651117268472003
2 2.647420653639688
3 2.549778604025036
4 2.5390340228045623
5 2.454311205465264
6 0.7219081899892659
7 7.848649884278856
8 1.6983979201850683
9 3.690232890011628
10 57.763332607827785
11 61.419318608650144
12 0.14702468714141928
Updating best MAPE to 0.14702468714141928...
Updating best seed to 12...
13 3.0769156219571165
14 2.112526835051635
15 1.2968491679018976
16 1.2716767351922784
17 3.275891020036979
18 0.6738698328144814
19 1.3888232340689326
20 0.7102218140024317
21 5.789048119187525
22 3.2926935335038885
23 1.2347349865162491
24 0.15155429043203855
25 4.249822673127027
26 3.7111258351224166
27 61.4492801399316
28 2.411963093733344
29 3.7007788543267033
30 1.4736613769933768
31 3.7342144063688365
32 0.5232756412306216
33 0.8610876032295154
34 2.2145188536514504
35 3.3104691295916737
36 1.9865533225800784
37 5.630104476257179
38 0.9545287894162244
39 5.931859979816

# ConvLSTM

In [20]:
from tensorflow.keras.layers import ConvLSTM2D

# Number of subsequences to break X into (we do 15 = 5x3, 5 subsequences of size 3 each)
nSeq = 3
nSteps = 2
# Each input is rows x columns, we have rows=1 and columns=nSteps

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainConv = XTrainList.reshape((XTrainList.shape[0], nSeq, 1, nSteps, nFeatures))
# XValidConv = XValidList.reshape((XValidList.shape[0], nSeq, 1, nSteps, nFeatures))
# XTestConv = XTestList.reshape((XTestList.shape[0], nSeq, 1, nSteps, nFeatures))

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
    model.add(Flatten())
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')
    
    # fit model
    model.fit(XTrainConv, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidListNew = XValidList.copy()
    for day in range(k):
        XValidConv = XValidListNew.reshape((XValidListNew.shape[0], nSeq, 1, nSteps, nFeatures))
        yPred = model.predict(XValidConv, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidListNew = np.delete(XValidListNew, 0, axis=1)
        XValidListNew = np.append(XValidListNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
model.add(Flatten())
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainConv, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestListNew = XTestList.copy()
for day in range(k):
    XTestConv = XTestListNew.reshape((XTestListNew.shape[0], nSeq, 1, nSteps, nFeatures))
    yPred = model.predict(XTestConv, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestListNew = np.delete(XTestListNew, 0, axis=1)
    XTestListNew = np.append(XTestListNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 1.5915821649047603
Updating best MAPE to 1.5915821649047603...
Updating best seed to 0...
1 0.3075386479502843
Updating best MAPE to 0.3075386479502843...
Updating best seed to 1...
2 0.5028132600799115
3 12.024816192313228
4 195.47254647832455
5 2.9390445553416904
6 99.80604394722505
7 99.31889222563287
8 99.42663300478937
9 1.9329454973517035
10 1.5165840615410402
11 1.6120918099248343
12 0.8270486574397375
13 4.182829783820238
14 0.5113789512244111
15 0.5191911631902775
16 2.90932919948105
17 180.0299235769612
18 4086.9491847531053
19 99.83721307358087
20 1.335379230315361
21 2.798471160307777
22 0.23673055316090674
Updating best MAPE to 0.23673055316090674...
Updating best seed to 22...
23 0.427533378416414
24 1.4182820888932572
25 2.5478595181365664
26 1.097827505941807
27 5.9285985319245915
28 0.5941575849014415
29 0.7519250668465853
30 0.20149050665617682
Updating best MAPE to 0.20149050665617682...
Updating best seed to 30...
31 1.0641071606922585
32 6.9127896224824115
33 3.9

In [23]:
# import matplotlib.pyplot as plt
# from statsmodels.tsa.statespace.sarimax import SARIMAX

# # ARIMA
# def getPredictions(X, nDaysInFuture=5, invertible=False, pdqTuple = (1, 2, 2)):
#     p, d, q = pdqTuple
#     predList = []
#     Xcopy = X.copy()
#     for i in range(nDaysInFuture):
#         if invertible:
#             model = SARIMAX(Xcopy, order=(p, d, q))

#             model_fit = model.fit(disp=False)

#             # make prediction
#             yhat = model_fit.predict(len(Xcopy), len(Xcopy), typ='levels')
#         else:
#             model = SARIMAX(Xcopy, order=(p, d, q), enforce_invertibility=False)

#             model_fit = model.fit(disp=False)

#             # make prediction
#             yhat = model_fit.predict(len(Xcopy), len(Xcopy), typ='levels')
#         Xcopy = np.append(Xcopy, yhat)
#         predList.append(np.around(yhat[0]))
        
#     return predList

# yPredARIMA = getPredictions(nCasesGreaterDaysSinceList[-15-k:-k], nDaysInFuture=k)

# plt.figure(figsize=(10,10))
# datesForPlottingList = datesGreaterDaysSinceList[-k:]
# groundTruthList = nCasesGreaterDaysSinceList[-k:]

# if countryName == 'US':
#     plt.ylabel('Number of cases (in millions)', fontsize=20)
#     plt.plot(datesForPlottingList, np.divide(groundTruthList, 1e6), '-o', label='Ground Truth');
#     plt.plot(datesForPlottingList, np.divide(yPredVanillaLSTM[-1], 1e6), '-o', label='Vanilla LSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredStackedLSTM[-1], 1e6), '-o', label='Stacked LSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredBidirectionalLSTM[-1], 1e6), '-o', label='Bidirectional LSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredCNNLSTM[-1], 1e6), '-o', label='CNN LSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredConvLSTM[-1], 1e6), '-o', label='ConvLSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredConvLSTM[-1], 1e6), '-o', label='ConvLSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredARIMA, 1e6), '-o', label='ARIMA');
# else:
#     plt.ylabel('Number of cases', fontsize=20)
#     plt.plot(datesForPlottingList, groundTruthList, '-o', label='Ground Truth');
#     plt.plot(datesForPlottingList, yPredVanillaLSTM[-1], '-o', label='Vanilla LSTM');
#     plt.plot(datesForPlottingList, yPredStackedLSTM[-1], '-o', label='Stacked LSTM');
#     plt.plot(datesForPlottingList, yPredBidirectionalLSTM[-1], '-o', label='Bidirectional LSTM');
#     plt.plot(datesForPlottingList, yPredCNNLSTM[-1], '-o', label='CNN LSTM');
#     plt.plot(datesForPlottingList, yPredConvLSTM[-1], '-o', label='ConvLSTM');
#     plt.plot(datesForPlottingList, yPredConvLSTM[-1], '-o', label='ConvLSTM');
#     plt.plot(datesForPlottingList, yPredARIMA, '-o', label='ARIMA');
# plt.xlabel('Date', fontsize=20);